In [95]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
lines_dic = [
    {"cx1": 420, "cy1": 750, "cx2": 550, "cy2": 750},
    {"cx1": 550, "cy1": 650, "cx2": 620, "cy2": 650},
    {"cx1": 990, "cy1": 750, "cx2": 1090, "cy2": 750},
    {"cx1": 1030, "cy1": 700, "cx2": 1110, "cy2": 700},
    {"cx1": 1180, "cy1": 640, "cx2": 1210, "cy2": 640},
    {"cx1": 1420, "cy1": 730, "cx2": 1490, "cy2": 730},
]

T = 27  # Umbral
AREA_MIN = 1500  # Área mínima
TOLERANCIA = 30  # Tolerancia línea
DISTANCIA_MAX = 50  # Distancia para tracking
TIMEOUT = 30  # Frames de memoria


In [97]:
video = cv2.VideoCapture('trafico01.mp4')
frames = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    frames.append(frame)
video.release()


In [98]:
fondo = frames[0].astype(np.float64)
step = max(1, len(frames) // 100)
frames_usados = 1

for i in range(step, len(frames), step):
    fondo += frames[i].astype(np.float64)
    frames_usados += 1

fondo = (fondo / frames_usados).astype(np.uint8)


In [99]:
altura, ancho = frames[0].shape[:2]
mascara = np.zeros((altura, ancho), dtype=np.uint8)

puntos_roi = np.array([
    [200, 550],   
    [200, 900],     
    [1700, 900],  
    [1700, 550]   
], dtype=np.int32)

cv2.fillPoly(mascara, [puntos_roi], 255)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1080, 1920), dtype=uint8)

In [100]:
contadores = [0] * len(lines_dic)
vehiculos_activos = [dict() for _ in range(len(lines_dic))]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

In [101]:
def encontrar_vehiculo_cercano(cx, cy, vehiculos_dict):
    """Busca si existe un vehículo cercano"""
    for vid, (vx, vy, _) in vehiculos_dict.items():
        if np.sqrt((cx - vx)**2 + (cy - vy)**2) < DISTANCIA_MAX:
            return vid
    return None

In [102]:
for frame_idx, frame in enumerate(frames):
    diferencia = cv2.absdiff(frame, fondo)
    gris = cv2.cvtColor(diferencia, cv2.COLOR_BGR2GRAY)
    _, binario = cv2.threshold(gris, T, 255, cv2.THRESH_BINARY)
    binario = cv2.morphologyEx(binario, cv2.MORPH_CLOSE, kernel)
    binario = cv2.morphologyEx(binario, cv2.MORPH_OPEN, kernel)
    
    binario = cv2.bitwise_and(binario, binario, mask=mascara)
    
    contornos, _ = cv2.findContours(binario, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img = frame.copy()
    
    if frame_idx % TIMEOUT == 0:
        for idx_linea in range(len(lines_dic)):
            viejos = [vid for vid, (_, _, ult_frame) in vehiculos_activos[idx_linea].items() 
                     if frame_idx - ult_frame > TIMEOUT]
            for vid in viejos:
                del vehiculos_activos[idx_linea][vid]
    
    
    for idx, line in enumerate(lines_dic):
        cv2.line(img, (line["cx1"], line["cy1"]), (line["cx2"], line["cy2"]), (0, 0, 255), 3)
        cv2.putText(img, f'L{idx+1}: {contadores[idx]}', 
                   ((line["cx1"] + line["cx2"])//2 - 30, line["cy1"] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area > AREA_MIN:
            x, y, w, h = cv2.boundingRect(contorno)
            centro_x = x + w // 2
            centro_y = y + h // 2
            
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
            
            
            for idx, line in enumerate(lines_dic):
                if (line["cx1"] <= centro_x <= line["cx2"] and 
                    abs(centro_y - line["cy1"]) < TOLERANCIA):
                    
                    vid_existente = encontrar_vehiculo_cercano(centro_x, centro_y, 
                                                               vehiculos_activos[idx])
                    
                    if vid_existente is None:
                        nuevo_id = f"v_{frame_idx}_{centro_x}"
                        vehiculos_activos[idx][nuevo_id] = (centro_x, centro_y, frame_idx)
                        contadores[idx] += 1
                    else:
                        vehiculos_activos[idx][vid_existente] = (centro_x, centro_y, frame_idx)
    
    cv2.putText(img, f'Total: {sum(contadores)}', 
               (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)


    cv2.imshow('Conteo de Vehiculos', img)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [103]:
print("\n" + "="*50)
print("RESULTADOS FINALES")
print("="*50)
for idx, count in enumerate(contadores):
    print(f"Línea {idx+1}: {count:3d} vehículos")
print("-"*50)
print(f"TOTAL:    {sum(contadores):3d} vehículos")
print("="*50)


RESULTADOS FINALES
Línea 1:   2 vehículos
Línea 2:   0 vehículos
Línea 3:   1 vehículos
Línea 4:   2 vehículos
Línea 5:   0 vehículos
Línea 6:   0 vehículos
--------------------------------------------------
TOTAL:      5 vehículos
